In [ ]:
# Import packages
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import pprint
import os
import re

In [ ]:
# Create the folder for the obtained books
folderPath = 'project_gutenberg'

if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# Link to the target URL
url_booklists = 'https://www.gutenberg.org/browse/languages/zh'

res = req.get(url_booklists)

soup = bs(res.text, "lxml")

# Extract data from the HTML of the webpage with beautiful soup selector
for a in soup.select('li.pgdbetext a[href]')[0:260]:
    # Filterd out the non-Chinese books using the regular expression 
    English_book_title = re.match(r'[a-zA-Z]+', a.get_text())
    if not English_book_title:
        Chinese_book_title = re.sub(r'[=\\/*?:"<>|\r\n]', '', a.get_text()).strip()
        book_title = Chinese_book_title
        book_link = 'https://www.gutenberg.org' + a['href']
    
        book_id = re.search(r'\d+', book_link)[0]

        # link_article: the URL is based on each book's book id
        link_article = f'https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}-images.html'

        res_ = req.get(link_article)

        soup_ = bs(res_.text,"lxml")

        soup_.select_one('div.container').decompose()
        
        if soup_.select_one('p#id00000'):
            soup_.select_one('p#id00000').decompose()
        
    # Concatenating all the contents of one book
    content = ''
    
    for p in soup_.select('p'):
        content += p.get_text() + "\n" + "\n"

    # Save contents of one book with a .txt file, named with the book title 
    file_path = os.path.join(folderPath, f"{book_title}.txt")
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

        